In [1]:
#Part 8 - Introduction to Protocols

import torch as th
import syft as sy
hook = sy.TorchHook(th)

# IMPORTANT: Local worker should not be a client worker
hook.local_worker.is_client_worker = False

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'C:\ProgramData\Anaconda3\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.2.so'


In [2]:
@sy.func2plan(args_shape=[(1,)])
def inc1(x):
    return x + 1

@sy.func2plan(args_shape=[(1,)])
def inc2(x):
    return x + 1

@sy.func2plan(args_shape=[(1,)])
def inc3(x):
    return x + 1

protocol = sy.Protocol([("worker1", inc1), ("worker2", inc2), ("worker3", inc3)])

In [3]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
charlie = sy.VirtualWorker(hook, id="charlie")

In [4]:
workers = alice, bob, charlie

protocol.deploy(*workers)

<Protocol id:64908312136 owner:me resolved>
 - alice: [PointerPlan | me:98006409966 -> alice:82194955458]
 - bob: [PointerPlan | me:16049411065 -> bob:6831171708]
 - charlie: [PointerPlan | me:91427378313 -> charlie:75908173589]

In [5]:
x = th.tensor([1.0])
ptr = protocol.run(x)
ptr

send alice
move alice  ->  bob
move bob  ->  charlie


(Wrapper)>[PointerTensor | me:42562583334 -> charlie:85590592219]

In [6]:
ptr.get()

tensor([4.])

In [7]:
james = sy.VirtualWorker(hook, id="james")
protocol.send(james)


x = th.tensor([1.0]).send(james)
ptr = protocol.run(x)
ptr

send remote run request to james
send alice
move alice  ->  bob
move bob  ->  charlie


(Wrapper)>[PointerTensor | me:69371271414 -> james:87712369266]

In [8]:
ptr = ptr.get()
ptr

(Wrapper)>[PointerTensor | me:87712369266 -> charlie:6168104770]

In [9]:
ptr = ptr.get()
ptr

tensor([4.])

In [10]:
protocol = sy.Protocol([("worker1", inc1), ("worker2", inc2), ("worker3", inc3)])
protocol.tag('my_protocol')
protocol.send(james)

In [11]:
me = sy.hook.local_worker # get access to me as a local worker

In [12]:
responses = me.request_search(['my_protocol'], location=james)
responses

[[PointerProtocol | me:12102206053 -> james:75430187158]]

In [13]:
ptr_protocol = responses[0]

protocol_back = ptr_protocol.get()
protocol_back

<Protocol id:75430187158 owner:me>
 - me: <Plan inc1 id:82194955458 owner:me built>
 - me: <Plan inc2 id:6831171708 owner:me built>
 - me: <Plan inc3 id:75908173589 owner:me built>

In [14]:
protocol_back.deploy(alice, bob, charlie)

x = th.tensor([1.0])
ptr = protocol_back.run(x)
ptr.get()

send alice
move alice  ->  bob
move bob  ->  charlie


tensor([4.])